In [49]:
%pylab inline
import numpy as np

Populating the interactive namespace from numpy and matplotlib


## Méthodes et Outils de l'I.A. et de la R.O.

 Pendant les 4 prochaines séances, nous allons voir: 
 * algorithmes de recherches 
 * algorithmes pour les jeux
 * dynamiques multi-agents

### Références Bibliographiques

Dans ce cours je suis largement des chapitres de référence suivants: 
* Recherche heuristique dans les graphes d'états, Henry Farreny, In Panorama de l'IA, vol. 1
* Solving problems by Searching (Chapter 3) et Informed (Heuristic) Search Strategies (Chapter 4), Stuart Russel and Peter Norvig. In Artificial Intelligence, A Modern Approach. 


## Graphes d'états

On considère de manière générale le problème consistant à trouver un chemin de coût minimal pour aller d'un **état initial** à un **état but** dans un **graphe d'états**. 

* Passer d'un état à un autre à un **coût**. 
* Une solution est **optimale** si elle permet de passer de l'état initial à l'état but avec un coût minimal. 
* Un paramètre important permettant d'apprécier la difficulté d'un problème est le **facteur de branchement** du graphe d'état (le nombre de successeurs possibles à chaque état). 





La taille des problèmes rend parfois impossible la création exhaustive du graphe d'état en mémoire. 

* par exemple, le problème du taquin de taille $n\times n$ donne lieu à $(n^2)!$ états possible. 


In [50]:
tiles1 = np.random.permutation(range(0,16)) # 0 is the empty slot
tiles2 = np.random.permutation(range(0,16)) # 0 is the empty slot
puzzle1 = np.array(tiles1)
puzzle2 = np.array(tiles2)
puzzle1 = np.reshape(puzzle1,(4,4))
puzzle2 = np.reshape(puzzle2,(4,4))
print(puzzle1)
print(puzzle2)

[[ 4  0 14  5]
 [ 1 12  3  6]
 [ 2  8 10 11]
 [15  9 13  7]]
[[ 4  7  1  8]
 [ 0 14  9 13]
 [11  6 12 10]
 [ 2  3 15  5]]


### Critères de comparaison

Suivant (Russel & Norvig, 2003), Ces différentes approches peuvent se comparer en terme de :
* **complétude**: l'algorithme trouve-t-il une solution (si une existe)? 
* leur **qualité**: la solution trouvée est-elle optimale? 
* **complexité de calcul**: le temps de calcul nécessaire pour obtenir la solution. 
* **complexité en espace**: l'espace de stockage nécessaire pendant le calcul. 

## Rappels: Recherche non informée

Il existe plusieurs types possibles d'exploration, qui se distinguent selon la stratégie d'expansion des noeuds. 

* depth-first (profondeur d'abord): on étend le noeud le plus profond de le frontière. Noeuds à stocker: $O(bd)$
* breadth-first (largeur d'abord): tous les successeurs d'un noued sont étendus. Noeuds à stocker: $O(b^{d+1})$
* best-first (meilleur d'abord): on étend le noeud avec le coût de chemin le plus faible. 

## Recherche heuristique informée

Idée: utiliser des connaissances du domaine pour guider la recherche. 

### Algortihme $A^*$

L'algorithme $A^*$ est un algorithme best-first, qui étend en priorité le noeud de la frontière avec le meilleur score $f(n)$

$$ f(n) = g(n) + h(n) $$ où:
* $g(n)$ est le coût du chemin jusqu'au noeud $n$
* $h(n)$ est le coût (estimé) de $n$ jusqu'à un état but. 

### Propriétés des heuristiques


#### Admissibilité 
* une heuristique est **admissible** lorsqu'elle ne surestime jamais la distance à l'état but (on parle aussi d'heuristique minorante)

Sous l'hypothèse d'admissibiilité de l'heuristique, $A^*$ retourne une solution optimale. 




#### Consistence

* une heuristique est consistante ssi pour tout état $m$ et tout fils de $n$ de $m$, on a:
    $$ h(m)≤cost(m,n)+h(n)$$
Autrement dit, les valeurs de $f(n)$ sont non-décroissantes sur les chemins depuis la racine.


### Implémentation

Gestion de la frontière grâce à un tas (permettant l'insertion rapide et garantissant que l'élement racine est toujours celui de meilleure priorité)

In [54]:
import heapq

q=[]
heapq.heappush(q,2)
heapq.heappush(q,5)
heapq.heappush(q,3)
heapq.heappush(q,1)
h = heapq.heappop(q)
print (q)
print(h)


[2, 5, 3]
1


### Problème de ré-expansion de noeuds

Important: si l'heuristique utilisée est consistante, alors un noeud un sommet développé par $A^*$ ne peut l'être qu'une fois. Autrement dit, les noeuds qui passent en réserve ne repassent jamais dans la frontière. 

Lorque l'heuristique n'est pas consistante, ce n'est pas le cas et on peut avoir à ré-étendre des noeuds... 


In [ ]:
def astar(p):
    """ application de l'algorithme a-star sur un probleme donné
        """
    nodeInit = Noeud(p.init,0,None)
    frontiere = [(nodeInit.g+p.h_value(nodeInit.etat,p.but),nodeInit)] 
    reserve = {}        
    bestNoeud = nodeInit
    
    while frontiere != [] and not p.estBut(bestNoeud.etat):              
        (min_f,bestNoeud) = heapq.heappop(frontiere)         
    # Suppose qu'un noeud en réserve n'est jamais ré-étendu 
    # Hypothèse de consistence de l'heuristique
    # ne gère pas les duplicatas dans la frontière
    
        if p.immatriculation(bestNoeud.etat) not in reserve:            
            reserve[p.immatriculation(bestNoeud.etat)] = bestNoeud.g #maj de reserve
            nouveauxNoeuds = bestNoeud.expand(p)            
            for n in nouveauxNoeuds:
                f = n.g+p.h_value(n.etat,p.but)
                heapq.heappush(frontiere, (f,n))              
    # Afficher le résultat                    
    return

## Trouver les bonnes heuristiques

Notons tout d'abord que l'heuristique nulle est bien une heuristique admissible. Que se passe-t-il dans ce cas? 

Des heuristiques classiques pour les problèmes de navigation/recherche de chemin (path-finding) sont: 
* distances de Manhattan
* distance euclidienne 

Quelles heuristiques pour le taquin? 

### Pattern Databases

Idée: on s'intéresse au coût exact de sous-problèmes de notre problème général, qui consistuent des bornes inf du coût du problème général. 

On créé donc une base de donnée avec le coût exact correspondant à toutes les configurations possibles du sous-problème. 
Pour cela, on part du but et on calcule le coût de chaque configuration rencontrée. 





## Recherche dans les graphes d'états
Notions essentielles: A*, Iterative deepening
Ici on suit essentiellement H. Farreny, Chapitre 1 du panorama. 

* Cours: Recherche dans un graphe d'état (exemple du taquin?), 
 recherche non informée: Bread-first search, 
 recherche informée: A*
* TD: 
* TME:

## Recherche dans les graphes
MCTS

## Interaction stratégique: Jeux 
Prisoner Dilemma
minmax
vers le IPD localisé


## Dynamique de systèmes multiagents

Cours: 
* best-response dynamics
* fictitious play
* replicator dynamics? 
* (coalition?)

TD: reprendre exercice L3-IA

TME: best-response dynamics avec des agents qui cherchent des plus courts chemins dans un graphe (cf. Rovatsos)



In [9]:
import pygame